# 1.Introduction

目標：基礎的な最適化手法を試してみよう！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 最適化の可視化

同じディレクトリにある`visualizer.py`にあるクラスで可視化できるようにしました！（拍手）

In [ ]:
from visualizer import Field, Adventurer

In [ ]:
def x2_y2(x, y):
    return x ** 2 + y ** 2

# フィールドの定義
field = Field(
    x2_y2,   # フィールドの高さを決める関数
    (-1, 1),     # xの範囲
    (-0.5, 0.5)  # yの範囲
)

# フィールドをプロットする
field.plot()

In [ ]:
# 無駄に高機能なので3Dでプロットもできます
field.plot_3d()

## SGD (Stochastic Gradient Descent)

勾配 $dL/dW$ と学習率 $\eta$ によってパラメータを更新していく手法

$$
W \leftarrow W - \eta\frac{\partial L}{\partial W}
$$

In [ ]:
# SGDの実装
class SGD:
    def __init__(self, lr=0.1):
        self.lr = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]  # 学習率 * 勾配 だけパラメータの値を減らす

### 可視化してみよう！

In [ ]:
field = Field(
    x2_y2,   # z = x^2 + y^2
    (-1, 1), # xの範囲
    (-1, 1)  # yの範囲
)

In [ ]:
# adventurer：冒険者
# スタート地点を与える
adventurer = Adventurer(0.5, 0.5)

# パラメータを最適化させる
adventurer.optimize(field, SGD())

In [ ]:
# adventurerがたどった道筋を取得
route = adventurer.route

# routeをフィールドに表示
field.plot(route=route)

### SGDの弱点は？？

↓こんなとき

In [ ]:
# SGDに強い関数
def anti_SGD(x, y):
    return x ** 2 / 20 + y ** 2

In [ ]:
field = Field(anti_SGD, (-1, 1), (-0.5, 0.5))
adventurer = Adventurer(-0.5, 0.2)  # 冒険者の初期値は(-0.5, 0.2)
sgd = SGD(lr=1)  # 学習率を1に設定

In [ ]:
adventurer.optimize(field, sgd)
adventurer.route

In [ ]:
field.plot(adventurer.route)

## Momentum

**運動量**をもとにしたパラメータ更新手法です。式はこんな感じ

速度の更新
$$
v \leftarrow \alpha v - \eta\frac{\partial L}{\partial W}
$$

パラメータの更新
$$
W \leftarrow W + v
$$

物理好きの方ならピントくるのでは？？

In [ ]:
# Momentumの実装
class Momentum:
    def __init__(self, lr=0.1, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v == None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

In [ ]:
# SGDに強いフィールド
field = Field(anti_SGD, (-1, 1), (-0.5, 0.5))

# さっきのフィールドで可視化してみる
adventurer2 = Adventurer(-0.5, 0.2)

# momentumを使う
momentum = Momentum()

In [ ]:
# momentumで最適化
adventurer2.optimize(field, momentum)

adventurer2.route

In [ ]:
field.plot(adventurer2.route)